# Graph Embeddings

## Introduction
I recently learned that people are trying to use deep learning to solve problems from graph theory after a friend linked me the paper, [Learning Combinatorial Optimization Algorithms over Graphs](https://arxiv.org/pdf/1704.01665.pdf). I find this subject very interesting and want to explore it further. In this paper, I will try to create some simple graph representations by using vertex embeddings and a neural network. I am going to try to create what is essentially a lookup table using deep learning.

This is also a great exercise for me to learn how to use PyTorch


## Data
To create the graph, I will use a simple dictionary of nodes that contains a set of the neighbors. I will also define a next function so we can get batches of data to train the neural network.

In [1]:
from fastai.model import fit

In [2]:
from random import sample, randint, choice

In [3]:
import numpy as np

In [4]:
from torch import Tensor as T
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
from torch import optim

In [302]:
class Graph:
    def __init__(self, n_vert = 10, n_neigh = 3, bs=64):
        """
        Initialize a graph object. Note that graph is not guaranteed to be
        fully connected
        
        TODO:
        Graph connectivity visualization
        """
        self.bs = bs
        self.vertices = range(n_vert)
        self.graph = {}
        for v in self.vertices:
            possible = set(self.vertices).difference({v}) # remove loops
            edges = set(sample(possible, n_neigh)) # add n neighbors to each vertex
            self.graph[v] = edges
            
        for v in self.vertices:
            for n in self.graph[v]:
                self.graph[n].add(v)
            
    def pos_sample(self):
        v = choice(self.vertices)
        x = [v, choice(tuple(self.graph[v]))]
        return x
    
    def neg_sample(self):
        v = choice(self.vertices)
        possible = set(self.vertices).difference(self.graph[v])
        possible = possible.difference({v})
        x = [v, choice(tuple(possible))]
        return x
    
    def __iter__(self, bs=64):
        """
        Set the batch size
        
        TODO: put this in init
        """
        self.bs = bs
        return self
        
    def __next__(self):
        """
        Returns a batch of data for use in training. The vertices are randomly chosen
        so it is not guaranteed that all vertices are in each training step - or that
        every possible neighbor will be chosen. Positive and negative examples are 
        one hot encoded for simplicity
        """
        n_pos = int(self.bs / 2)
        n_neg = self.bs - n_pos
        
        pos = [[self.pos_sample(), [1, 0]] for _ in range(n_pos)]
        neg = [[self.neg_sample(), [0, 1]] for _ in range(n_neg)]
        X, Y = zip(*(pos + neg))
        X = list(zip(*X))
        return X, Y

In [303]:
n_vert = 10
n_neigh = 3
bs = 64
graph = Graph(n_vert=n_vert, n_neigh=n_neigh, bs=bs)

In [304]:
graph.graph

{0: {1, 3, 4, 6, 7, 9},
 1: {0, 4, 6, 7, 9},
 2: {4, 6, 7, 8, 9},
 3: {0, 4, 5, 6, 8},
 4: {0, 1, 2, 3, 5, 6},
 5: {3, 4, 6, 7, 8, 9},
 6: {0, 1, 2, 3, 4, 5, 7, 8},
 7: {0, 1, 2, 5, 6},
 8: {2, 3, 5, 6, 9},
 9: {0, 1, 2, 5, 8}}

In [305]:
# X, Y = next(graph)

In [306]:
# %timeit next(graph) # data is generated at 5,500 times per second. speed should not be an issue

In [149]:
class Model(nn.Module):
    def __init__(self, n_vert, n_neigh, nh=10, p1=0.2, p2=0.2):
        super().__init__()
        
        emb_dim = 5 # dimension of the vertex embedding
        
        self.v = nn.Embedding(n_vert, emb_dim)
        self.v.weight.data.uniform_(-0.01, 0.01)
#         self.v.weight.data.uniform_(-1, 1)
        
        self.lin1 = nn.Linear(emb_dim*2, nh)
        self.lin2 = nn.Linear(nh, 2)
        self.drop1 = nn.Dropout(p1)
        self.drop2 = nn.Dropout(p2)
        
    def forward(self, vertices):
        start = vertices[0,:]
        end = vertices[1,:]
        x = self.drop1(torch.cat([self.v(start), self.v(end)], dim=1))
        x = self.drop2(F.relu(self.lin1(x)))
        x = F.softmax(self.lin2(x), dim=1)
        return x

In [307]:
class Model(nn.Module):
    def __init__(self, n_vert, n_neigh, nh=10, p1=0.05, p2=0.05, p3=0.05):
        super().__init__()
        
        emb_dim = 5 # dimension of the vertex embedding
        
        self.v = nn.Embedding(n_vert, emb_dim)
        self.v.weight.data.uniform_(-0.01, 0.01)
        
        self.lin1 = nn.Linear(emb_dim, nh)
        self.lin2 = nn.Linear(nh*2, nh)
        self.lin3 = nn.Linear(nh, 2)
        
        self.drop1 = nn.Dropout(p1)
        self.drop2 = nn.Dropout(p2)
        self.drop3 = nn.Dropout(p3)
        
    def forward(self, vertices):
        start = vertices[0,:]
        end = vertices[1,:]
        x = F.relu(self.lin1(self.v(start)))
        y = F.relu(self.lin1(self.v(end)))
        x = self.drop1(torch.cat([x, y], dim=1))
        x = self.drop2(F.relu(self.lin2(x)))
        x = F.softmax(self.lin3(x), dim=1)
        return x

In [308]:
X, Y = next(graph)
inp = Variable(torch.LongTensor(X))

In [309]:
model = Model(n_vert, n_neigh).cuda()

In [310]:
wd=1e-3
optimizer = optim.Adam(model.parameters(), 1e-3, weight_decay=wd)
criterion = nn.CrossEntropyLoss()

In [311]:
running_loss = 0.0
running_corrects = 0

In [312]:
model.v.weight

Parameter containing:
1.00000e-03 *
 -0.0495  2.9260 -6.1389 -6.4662  5.7728
  0.0171  0.9879 -6.2533 -5.8708 -4.8033
 -4.8220 -7.9444 -5.3494 -2.6285 -9.2742
  1.0761 -9.0331  6.1081  8.2885  1.0739
  9.6744 -9.0215 -3.5673  0.3676 -7.1181
  3.4299 -5.8806  5.2835 -6.2741  2.7986
  7.1504  7.7714 -6.4761  2.8440 -1.2669
 -1.8374 -0.7353  2.1099  6.0447 -3.1118
 -2.1981  1.5886 -6.9561  0.7860  2.0235
 -7.2763  9.1767 -4.7947  4.5776  1.3037
[torch.cuda.FloatTensor of size 10x5 (GPU 0)]

In [317]:
for i in range(3000):
    # get batch of data
    X, Y = next(graph)
    inputs = Variable(torch.LongTensor(X)).cuda()
    labels = Variable(torch.LongTensor(Y)).cuda()
    labels = torch.max(labels, dim=1)[1]

    # zero gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    model.v.weight

    running_loss += loss.data[0]
    if i % 1000 == 999:    # print every 100 mini-batches
        print('[%d, %5d] loss: %.3f' % (1, i + 1, running_loss / 2000))
        running_loss = 0.0

[1,   100] loss: 0.027
[1,   200] loss: 0.027
[1,   300] loss: 0.026
[1,   400] loss: 0.026
[1,   500] loss: 0.026
[1,   600] loss: 0.026
[1,   700] loss: 0.026
[1,   800] loss: 0.026
[1,   900] loss: 0.026
[1,  1000] loss: 0.026
[1,  1100] loss: 0.026
[1,  1200] loss: 0.026
[1,  1300] loss: 0.026
[1,  1400] loss: 0.026
[1,  1500] loss: 0.025
[1,  1600] loss: 0.025
[1,  1700] loss: 0.025
[1,  1800] loss: 0.025
[1,  1900] loss: 0.025
[1,  2000] loss: 0.025
[1,  2100] loss: 0.025
[1,  2200] loss: 0.025
[1,  2300] loss: 0.025
[1,  2400] loss: 0.025
[1,  2500] loss: 0.025
[1,  2600] loss: 0.025
[1,  2700] loss: 0.025
[1,  2800] loss: 0.025
[1,  2900] loss: 0.025
[1,  3000] loss: 0.025


In [314]:
model.v.weight

Parameter containing:
-0.1180 -0.1039  0.1714 -0.0076  0.1040
-0.0192 -0.0215 -0.0070 -0.0905  0.0272
 0.0734  0.0600 -0.1057 -0.0196 -0.0672
-0.0217 -0.0284  0.0179 -0.0655  0.0341
-0.4502 -0.4815  0.3989 -0.5791  0.4789
 0.0676  0.0581 -0.0678  0.0644 -0.0660
-0.9655 -0.9808  0.9141 -1.0030  1.0020
 0.3739  0.3812 -0.3967  0.4856 -0.3915
 0.3412  0.3353 -0.3480  0.3213 -0.3450
 0.7610  0.7670 -0.7750  0.7438 -0.7818
[torch.cuda.FloatTensor of size 10x5 (GPU 0)]

In [315]:
model.drop1.p = 0
model.drop2.p = 0

total = 0
correct = 0

for i in range(10):
    # get batch of data
    X, Y = next(graph)
    inputs = Variable(torch.LongTensor(X)).cuda()
    labels = Variable(torch.LongTensor(Y)).cuda()
    labels = torch.max(labels, dim=1)[1]
    
    _, preds = torch.max(model(inputs), dim=1)
    total += labels.size()[0]
    correct += (preds == labels).sum().data[0]
    
model.drop1.p = .2
model.drop2.p = .2

In [316]:
correct / total

0.853125